This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Introduction to Keras and TensorFlow

## What's TensorFlow?

## What's Keras?

## Keras and TensorFlow: A brief history

## Setting up a deep-learning workspace

### Jupyter notebooks: The preferred way to run deep-learning experiments

### Using Colaboratory

#### First steps with Colaboratory

#### Installing packages with pip

#### Using the GPU runtime

## First steps with TensorFlow

#### Constant tensors and variables

**All-ones or all-zeros tensors**

In [2]:
# page 101
import tensorflow as tf
x = tf.ones(shape=(2, 1))
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)


In [3]:
x = tf.zeros(shape=(2, 1))
print(x)

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


**Random tensors**

In [4]:
x = tf.random.normal(shape=(3, 1), mean=0., stddev=1.)
print(x)

tf.Tensor(
[[-0.01102383]
 [ 0.1737031 ]
 [-0.35971305]], shape=(3, 1), dtype=float32)


In [5]:
x = tf.random.uniform(shape=(3, 1), minval=0., maxval=1.)
print(x)

tf.Tensor(
[[0.2175926 ]
 [0.28620458]
 [0.9183346 ]], shape=(3, 1), dtype=float32)


**NumPy arrays are assignable**

In [6]:
import numpy as np
x = np.ones(shape=(2, 2))
x[0, 0] = 0.

In [7]:
# causes error
# x = tf.ones(shape=(2, 2))
# x[0, 0] = 0.

**Creating a TensorFlow variable**

In [8]:
# page 102
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.59550786],
       [-0.7777252 ],
       [ 1.0618935 ]], dtype=float32)>


**Assigning a value to a TensorFlow variable**

In [9]:
v.assign(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>

**Assigning a value to a subset of a TensorFlow variable**

In [10]:
v[0, 0].assign(3.)

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

**Using `assign_add`**

In [11]:
v.assign_add(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [2.],
       [2.]], dtype=float32)>

#### Tensor operations: Doing math in TensorFlow

**A few basic math operations**

In [12]:
a = tf.ones((2, 2))
b = tf.square(a)
c = tf.sqrt(a)
d = b + c
e = tf.matmul(a, b)
e *= d

#### A second look at the GradientTape API

**Using the `GradientTape`**

In [13]:
input_var = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
   result = tf.square(input_var)
gradient = tape.gradient(result, input_var)
print(input_var)
print(gradient)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>
tf.Tensor(6.0, shape=(), dtype=float32)


**Using `GradientTape` with constant tensor inputs**

In [14]:
input_const = tf.constant(3.)
with tf.GradientTape() as tape:
   tape.watch(input_const)
   result = tf.square(input_const)
gradient = tape.gradient(result, input_const)
print(input_const)
print(gradient)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


**Using nested gradient tapes to compute second-order gradients**

In [15]:
time = tf.Variable(0.)
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        position =  4.9 * time ** 2
    speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)
print(speed)
print(acceleration)
print(position)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(9.8, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)


#### An end-to-end example: A linear classifier in pure TensorFlow

**Generating two classes of random points in a 2D plane**

In [16]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(
    mean=[0, 3],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(
    mean=[3, 0],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class)

**Stacking the two classes into an array with shape (2000, 2)**

In [17]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
inputs

array([[-0.12625094,  1.9908315 ],
       [ 2.0564382 ,  5.244576  ],
       [-1.361273  ,  2.1515448 ],
       ...,
       [ 4.329411  ,  1.4080533 ],
       [ 3.6324203 , -1.8510112 ],
       [ 2.9110565 ,  0.1225234 ]], dtype=float32)

**Generating the corresponding targets (0 and 1)**

In [18]:
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"),
                     np.ones((num_samples_per_class, 1), dtype="float32")))

**Plotting the two point classes**

In [19]:
import matplotlib.pyplot as plt
plt.scatter(inputs[:, 0], inputs[:, 1], c=targets[:, 0])
# plt.show()
plt.savefig("../pngs/3ch/ovals.png")
plt.close()

**Creating the linear classifier variables**

In [20]:
input_dim = 2
output_dim = 1
W = tf.Variable(initial_value=tf.random.uniform(shape=(input_dim, output_dim)))
b = tf.Variable(initial_value=tf.zeros(shape=(output_dim,)))

**The forward pass function**

In [21]:
def model(inputs):
    return tf.matmul(inputs, W) + b

**The mean squared error loss function**

In [22]:
def square_loss(targets, predictions):
    per_sample_losses = tf.square(targets - predictions)
    return tf.reduce_mean(per_sample_losses)

**The training step function**

In [23]:
learning_rate = 0.1

def training_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = square_loss(targets, predictions)
    grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b])
    W.assign_sub(grad_loss_wrt_W * learning_rate)
    b.assign_sub(grad_loss_wrt_b * learning_rate)
    return loss

**The batch training loop**

In [24]:
for step in range(40):
    loss = training_step(inputs, targets)
    print(f"Loss at step {step}: {loss:.4f}")

Loss at step 0: 3.4262
Loss at step 1: 0.5719
Loss at step 2: 0.1989
Loss at step 3: 0.1383
Loss at step 4: 0.1219
Loss at step 5: 0.1123
Loss at step 6: 0.1044
Loss at step 7: 0.0973
Loss at step 8: 0.0908
Loss at step 9: 0.0849
Loss at step 10: 0.0796
Loss at step 11: 0.0747
Loss at step 12: 0.0703
Loss at step 13: 0.0663
Loss at step 14: 0.0626
Loss at step 15: 0.0593
Loss at step 16: 0.0563
Loss at step 17: 0.0535
Loss at step 18: 0.0510
Loss at step 19: 0.0488
Loss at step 20: 0.0467
Loss at step 21: 0.0448
Loss at step 22: 0.0431
Loss at step 23: 0.0415
Loss at step 24: 0.0401
Loss at step 25: 0.0388
Loss at step 26: 0.0376
Loss at step 27: 0.0366
Loss at step 28: 0.0356
Loss at step 29: 0.0347
Loss at step 30: 0.0339
Loss at step 31: 0.0332
Loss at step 32: 0.0325
Loss at step 33: 0.0319
Loss at step 34: 0.0314
Loss at step 35: 0.0309
Loss at step 36: 0.0304
Loss at step 37: 0.0300
Loss at step 38: 0.0296
Loss at step 39: 0.0293


In [25]:
predictions = model(inputs)
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5)
# plt.show()
plt.savefig("../pngs/3ch/preds.png")
plt.close()

In [26]:
x = np.linspace(-1, 4, 100)
y = - W[0] /  W[1] * x + (0.5 - b) / W[1]
plt.plot(x, y, "-r")
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5)
plt.savefig("../pngs/3ch/line.png")
plt.close()

## Anatomy of a neural network: Understanding core Keras APIs

### Layers: The building blocks of deep learning

#### The base Layer class in Keras

**A `Dense` layer implemented as a `Layer` subclass**

In [27]:
from tensorflow import keras

class SimpleDense(keras.layers.Layer):

    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units),
                                 initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros")

    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [28]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)

(2, 32)


#### Automatic shape inference: Building layers on the fly

In [29]:
from tensorflow.keras import layers
layer = layers.Dense(32, activation="relu")
print(layer)

In [30]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(32)
])

In [31]:
model = keras.Sequential([
    SimpleDense(32, activation="relu"),
    SimpleDense(64, activation="relu"),
    SimpleDense(32, activation="relu"),
    SimpleDense(10, activation="softmax")
])

### From layers to models

### The "compile" step: Configuring the learning process

In [32]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer="rmsprop",
              loss="mean_squared_error",
              metrics=["accuracy"])

In [33]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

### Picking a loss function

### Understanding the fit() method

**Calling `fit()` with NumPy data**

In [34]:
# page 115
history = model.fit(
    inputs,
    targets,
    epochs=5,
    batch_size=128
)

Epoch 1/5


16/16 [==============================] - 1s 3ms/step - loss: 5.5876 - binary_accuracy: 0.4520
Epoch 2/5
16/16 [==============================] - 0s 2ms/step - loss: 5.2466 - binary_accuracy: 0.4505
Epoch 3/5
16/16 [==============================] - 0s 2ms/step - loss: 4.9653 - binary_accuracy: 0.4475
Epoch 4/5
16/16 [==============================] - 0s 2ms/step - loss: 4.6958 - binary_accuracy: 0.4460
Epoch 5/5
16/16 [==============================] - 0s 2ms/step - loss: 4.4383 - binary_accuracy: 0.4430


In [35]:
history.history

{'loss': [5.587596893310547,
  5.246648788452148,
  4.965266227722168,
  4.695825576782227,
  4.438285827636719],
 'binary_accuracy': [0.4519999921321869,
  0.4505000114440918,
  0.44749999046325684,
  0.44600000977516174,
  0.4429999887943268]}

### Monitoring loss and metrics on validation data

**Using the `validation_data` argument**

In [36]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)

Epoch 1/5


88/88 [==============================] - 1s 6ms/step - loss: 0.1297 - binary_accuracy: 0.9550 - val_loss: 0.0469 - val_binary_accuracy: 0.9933
Epoch 2/5
88/88 [==============================] - 0s 4ms/step - loss: 0.0741 - binary_accuracy: 0.9564 - val_loss: 0.2146 - val_binary_accuracy: 0.8750
Epoch 3/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0758 - binary_accuracy: 0.9500 - val_loss: 0.2056 - val_binary_accuracy: 0.8283
Epoch 4/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0778 - binary_accuracy: 0.9493 - val_loss: 0.0313 - val_binary_accuracy: 0.9967
Epoch 5/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0685 - binary_accuracy: 0.9679 - val_loss: 0.0509 - val_binary_accuracy: 0.9850


In [37]:
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)
print(loss_and_metrics)

5/5 [==============================] - 0s 4ms/step - loss: 0.0509 - binary_accuracy: 0.9850
[0.050871577113866806, 0.9850000143051147]


### Inference: Using a model after training

In [38]:
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])

5/5 [==============================] - 0s 1ms/step
[[-0.14672685]
 [-0.30555427]
 [-0.3889262 ]
 [ 1.0086813 ]
 [ 0.6511597 ]
 [-0.18005568]
 [ 0.9536432 ]
 [ 0.17867693]
 [-0.05624068]
 [ 0.6070651 ]]


## Summary